# 12/5ゼミ

### 提案手法 AMR バージョン 
  
**Phase 1**  
区間重要度の中心を各手法で推定  
* $k$ 番目を除いた $n-1$ 個の $w_i$ に対して EV, GM, ALD で $w_{i}^{C}(k)$ を求める（この時点でそれらの総和は１となる）
* 上記で求めた $w_{i}^{C}(k)$ の和を $\mu_k$ 倍することで、 $k$ 番目の区間重要度中心は $1-\mu_k$ となる

**Phase 2**  
各 $k$ に対して、以下の線形最適化問題を解き、最適値を $d_{\overline{k}}^{*}$ とする：  
$$
\begin{align*}
\operatorname{minimize} ~~ & ~~
d_{\overline{k}} = \sum_{j \neq k} l_j, \\
\text{subject to} ~~ & ~~
a_{ij} \left( \mu w_{j}^{C} - l_j \right) \leq \mu w_{i}^{C} + l_i, &&
i \neq j, k, ~~ & j \neq k \\ & ~~
a_{kj} \left( \mu w_{j}^{C} - l_j \right) \leq 1-\mu + l_k, &&
j \neq k \\ & ~~
a_{ik} \{ \left(1- \mu \right) - l_k \} \leq \mu w_{i}^{C} + l_i, &&
i \neq k \\ & ~~
\left( 1 - \mu \right) + l_k + \sum_{i \neq j, k} \left( \mu w_{i}^{C} + l_i \right) + \mu w_{j}^{C} - l_j \geq 1, &&
i \in N \backslash k \\ & ~~
\left( 1 - \mu \right) - l_k + \sum_{i \neq j, k} \left( \mu w_{i}^{C} - l_i \right) + \mu w_{j}^{C} + l_j \leq 1, &&
i \in N \backslash k \\ & ~~
\mu w_{i}^{C} - l_i \geq \epsilon, &&
i \in N, ~~ & i \neq k \\ & ~~

\left( 1 - \mu \right) - l_k \geq \epsilon \\ & ~~
\sum_{i \neq k} \left( \mu w_{i}^{C} + l_i \right) + \left( 1 - \mu \right) - l_k \geq 1 \\ & ~~
\sum_{i \neq k} \left( \mu w_{i}^{C} - l_i \right) + \left( 1 - \mu \right) + l_k \leq 1 \\ & ~~


\end{align*}
$$



**Phase 3**  
各 $k$ に対して、以下の線形最適化問題を解き、最適値を $l_{k}^{*}$ とする：  
$$
\begin{align*}
\operatorname{minimize} ~~ & ~~
l_k, \\
\text{subject to} ~~ & ~~
a_{ij} \left( \mu w_{j}^{C} - l_j \right) \leq \mu w_{i}^{C} + l_i, &&
i \neq j, k, ~~ & j \neq k \\ & ~~
a_{kj} \left( \mu w_{j}^{C} - l_j \right) \leq 1-\mu + l_k, &&
j \neq k \\ & ~~
a_{ik} \{ \left(1- \mu \right) - l_k \} \leq \mu w_{i}^{C} + l_i, &&
i \neq k \\ & ~~
\left( 1 - \mu \right) + l_k + \sum_{i \neq j, k} \left( \mu w_{i}^{C} + l_i \right) + \mu w_{j}^{C} - l_j \geq 1, &&
i \in N \backslash k \\ & ~~
\left( 1 - \mu \right) - l_k + \sum_{i \neq j, k} \left( \mu w_{i}^{C} - l_i \right) + \mu w_{j}^{C} + l_j \leq 1, &&
i \in N \backslash k \\ & ~~
\mu w_{i}^{C} - l_i \geq \epsilon, &&
i \in N, ~~ & i \neq k \\ & ~~

\left( 1 - \mu \right) - l_k \geq \epsilon \\ & ~~
\sum_{i \neq k} \left( \mu w_{i}^{C} + l_i \right) + \left( 1 - \mu \right) - l_k \geq 1 \\ & ~~
\sum_{i \neq k} \left( \mu w_{i}^{C} - l_i \right) + \left( 1 - \mu \right) + l_k \leq 1 \\ & ~~
\sum_{i \neq j, k} l_k = d_{\overline{k}}^{*}\\ & ~~


\end{align*}
$$

**Phase 4**  
Phase 3 で導かれた結果は各$k$に対して存在するから、以下のようにして結果を統合する：  
* 各$l_{k}^{*}$に対して、  
$$\left\{\begin{array}{l}w_i(k)=\left[\mu_{k}^* w_i^c-l_{k}^{*},\:\: \mu_{k}^* w_i^c+l_{k}^{*}\right], \quad i \neq k \\ w_i(k)=\left[\left(1-\mu_{k}^*\right)-l_{k}^{*},\:\:\left(1-\mu_{k}^*\right)+l_{k}^{*}\right], \quad i=k\end{array}\right.$$
  

$$\overline{w}_i^L=\frac{1}{n} \sum_{i \in N} w_i^L(k) \quad \overline{w}_i^U=\frac{1}{n} \sum_{i \in N} w_i^U(k)$$  

この結果は，正規性を満たしている．

In [9]:
# import Pkg
# Pkg.add("DataFrames")
# Pkg.add("CSV")
# Pkg.add("IntervalSets")
# Pkg.add("LatexStrings")

In [10]:
using DataFrames, CSV
using IntervalArithmetic
using LaTeXStrings
using Statistics
# using Base
include("./libs/display-latex.jl")
include("./libs/crisp-pcm.jl")
include("./libs/analysis-indicators.jl")
include("./libs/solve-deterministic-ahp.jl")
include("./libs/AMR.jl")

AMR (generic function with 1 method)

In [11]:
# 評価基準数
N = 6;
# 真の区間重要度のセッティング
true_importance = "E"; # ["A", "B", "C", "D", "E"]

In [12]:
# 真の区間重要度を読み込み
df = CSV.File("./data/N"*string(N)*"_"*true_importance*"_Given_interval_weight.csv", header=false) |> DataFrame
T = Vector{Interval{Float64}}() # Float64のところにTを使うとおかしくなるのはなぜか？
for i in 1:N
    push!(T, interval(df[1, 2*i-1], df[1, 2*i]))
end

display(L"T = %$(intervalVectorLaTeXString(T))")

L"$T = \begin{pmatrix} \left[ 0.2, 0.26 \right] \\ \left[ 0.15, 0.25 \right] \\ \left[ 0.11, 0.25 \right] \\ \left[ 0.08, 0.22 \right] \\ \left[ 0.08, 0.18 \right] \\ \left[ 0.08, 0.14 \right] \end{pmatrix}$"

In [13]:
# 一対比較行列を1000個読み込み
df1 = CSV.File("./data/N"*string(N)*"_a3_"*true_importance*"_PCM_int.csv", header=false) |> DataFrame

# データフレームをN×Nのサブデータフレームに分割
subdfs = split_dataframe(df1, N);

In [14]:
# 結果格納用
P = Matrix{Float64}(undef, N, 1000)
Q = Matrix{Float64}(undef, N, 1000)
R = Matrix{Float64}(undef, N, 1000)
F = Matrix{Float64}(undef, N, 1000)

# 1000個のデータで推定
for (i, subdf) in enumerate(subdfs)

    # 一対比較行列を取り出す
    A = Matrix(subdf)
    # display(L"A = %$(matrixLaTeXString(A))")
    # if i == 779
    #     println("i = ", i)
    #     display(L"A = %$(matrixLaTeXString(A))")
    # end

    # 提案手法で区間重要度推定
    solution = AMR(A, EV)
    E = solution.W # 区間重要度の推定値
    # display(L"E = %$(intervalVectorLaTeXString(E))")
    # display(L"T = %$(intervalVectorLaTeXString(T))")

    TcapE = T .∩ E # 真の区間重要度と推定値の積集合
    TcupE = T .∪ E # 真の区間重要度と推定値の和集合
    # display(L"T ∩ E = %$(intervalVectorLaTeXString(TcapE))")
    # display(L"T ∪ E = %$(intervalVectorLaTeXString(TcupE))")
    
    # P, Q, R, F の計算
    P[:, i] = calculate_P(T, E) # 真の区間重要度と推定値の積集合の割合
    Q[:, i] = calculate_Q(T, E) # 真の区間重要度と推定値の積集合の割合
    R[:, i] = calculate_R(T, E) # 真の区間重要度と推定値の積集合の割合
    F[:, i] = calculate_F(T, E) # F値
    # println("i = ", i)
    # display(L"P = %$(VectorLaTeXString(P[:,i]))")
    # display(L"Q = %$(VectorLaTeXString(Q[:,i]))")
    # display(L"R = %$(VectorLaTeXString(R[:,i]))")
    # display(L"F = %$(VectorLaTeXString(F[:,i]))")
end

In [15]:
P_ave = mean(P)
Q_ave = mean(Q)
R_ave = mean(R)
F_ave = mean(F)
display(L"Pₐᵥₑ = %$(round(P_ave, digits=5))")
display(L"Qₐᵥₑ = %$(round(Q_ave, digits=5))")
display(L"Rₐᵥₑ = %$(round(R_ave, digits=5))")
display(L"Fₐᵥₑ = %$(round(F_ave, digits=5))")

L"$Pₐᵥₑ = 0.6118$"

L"$Qₐᵥₑ = 0.75162$"

L"$Rₐᵥₑ = 0.78664$"

L"$Fₐᵥₑ = 0.74093$"

In [16]:
P_var = var(P)
Q_var = var(Q)
R_var = var(R)
F_var = var(F)
display(L"Pᵥₐᵣ = %$(round(P_var, digits=5))")
display(L"Qᵥₐᵣ = %$(round(Q_var, digits=5))")
display(L"Rᵥₐᵣ = %$(round(R_var, digits=5))")
display(L"Fᵥₐᵣ = %$(round(F_var, digits=5))")

L"$Pᵥₐᵣ = 0.03416$"

L"$Qᵥₐᵣ = 0.04655$"

L"$Rᵥₐᵣ = 0.0416$"

L"$Fᵥₐᵣ = 0.02615$"